In [1]:
import os
import random
import shutil
import time
import warnings

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim

import torch.utils.data
from torch.utils.tensorboard import SummaryWriter
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Utiliy Functions.
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)
class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [3]:
# Ensure that we get deterministic results.
SEED=1
random.seed(SEED)
torch.manual_seed(SEED)
cudnn.deterministic = True

In [4]:
# Setup TensorBoard
writer = SummaryWriter(log_dir="/data/logs")

In [5]:
# Check that we have a CUDA enabled device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [6]:
MEAN_RGB = [0.47889522, 0.47227842, 0.43047404]
STD_RGB = [0.229, 0.224, 0.225]
IMG_SIZE = 128

In [7]:
# Load train and val dataset. 
TRAINDIR = "/data/train" # ImageNet train.
VALDIR = "/data/val" # ImageNet val.
BATCH_SIZE = 64
TRAIN_WORKERS = 6
VAL_WORKERS = 0
SHUFFLE = False

# Scale to ImageNet mean and STD since we will be using a model pretrained on ImageNet. 
transform_train = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(MEAN_RGB, STD_RGB),
])

transform_val = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(MEAN_RGB, STD_RGB),
])

# Load training data.
train_dataset = datasets.ImageFolder(
    TRAINDIR, transform=transform_train)

train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=BATCH_SIZE, num_workers=TRAIN_WORKERS, shuffle=SHUFFLE, pin_memory=True, sampler=None)

# Load validation data.
val_dataset = datasets.ImageFolder(
    VALDIR, transform=transform_val)

val_loader = torch.utils.data.DataLoader(
        val_dataset, batch_size=BATCH_SIZE, num_workers=VAL_WORKERS, shuffle=SHUFFLE, pin_memory=True, sampler=None)

In [8]:
NUM_CLASSES = 1000
ARCH = 'resnet18'
LR = 1e-3
MOMENTUM = 0.9
WEIGHT_DECAY = 1e-4
# Load model from PyTorch.
model = models.__dict__[ARCH]()
inf = model.fc.in_features
# Set fully connected layer to train with 1000 classes.
model.fc = nn.Linear(inf, NUM_CLASSES)
model.to(device)
# Setup optimizer and loss function. 
optimizer = torch.optim.SGD(model.parameters(), LR,
                                momentum=MOMENTUM,
                                weight_decay=WEIGHT_DECAY)
criterion = nn.CrossEntropyLoss().to(device)

In [9]:
PRINT_FREQ = BATCH_SIZE - 1
global_step = 0

In [10]:
# Define train step.
def train(train_loader, model, criterion, optimizer, epoch):
    global global_step
    # Keep progress of metrics.
    batch_time = AverageMeter('Time', ':6.3f')
    data_time = AverageMeter('Data', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(train_loader),
        [batch_time, data_time, losses, top1, top5],
        prefix="Epoch: [{}]".format(epoch))

    # Switch to train mode
    model.train()

    end = time.time()
    for i, (images, target) in enumerate(train_loader):
        # Measure data loading time.
        data_time.update(time.time() - end)
        
        # Move data to device
        images = images.to(device, non_blocking=True)
        target = target.to(device, non_blocking=True)
            
        # Compute output.
        output = model(images)
        loss = criterion(output, target)

        # Measure accuracy and record loss.
        acc1, acc5 = accuracy(output, target, topk=(1, 5))
        losses.update(loss.item(), images.size(0))
        top1.update(acc1[0], images.size(0))
        top5.update(acc5[0], images.size(0))
        
        # Write tensorboard logs.
        writer.add_scalar("Loss/train", loss, global_step=global_step)
        writer.add_scalar("Acc1/train", top1.avg, global_step=global_step)
        writer.add_scalar("Acc5/train", top5.avg, global_step=global_step)
        global_step += 1

        # Compute gradient.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Measure elapsed time.
        batch_time.update(time.time() - end)
        end = time.time()

        if i % PRINT_FREQ == 0:
            progress.display(i)

In [11]:
# Define validation step.
def validate(val_loader, model, criterion):
    global global_step
    # Keep progress of metrics.
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(val_loader),
        [batch_time, losses, top1, top5],
        prefix='Test: ')

    # Switch to evaluate mode.
    model.eval()

    with torch.no_grad():
        end = time.time()
        for i, (images, target) in enumerate(val_loader):
            
            # Move data to GPU if CUDA device is available.
            images = images.to(device, non_blocking=True)
            target = target.to(device, non_blocking=True)

            # Compute output.
            output = model(images)
            loss = criterion(output, target)

            # Measure accuracy and record loss.
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            losses.update(loss.item(), images.size(0))
            top1.update(acc1[0], images.size(0))
            top5.update(acc5[0], images.size(0))
            

            # Measure elapsed time.
            batch_time.update(time.time() - end)
            end = time.time()

            if i % PRINT_FREQ == 0:
                progress.display(i)

        print(' * Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'
              .format(top1=top1, top5=top5))
    
     # Write tensorboard logs.
    writer.add_scalar("Loss/val", loss, global_step=global_step)
    writer.add_scalar("Acc1/val", top1.avg, global_step=global_step)
    writer.add_scalar("Acc5/val", top5.avg, global_step=global_step)
    
    return top1.avg

In [12]:
for epoch in range(1):
    # train for one epoch
    train(train_loader, model, criterion, optimizer, epoch)

    # evaluate on validation set
    validate(val_loader, model, criterion)
    

Epoch: [0][    0/20019]	Time  0.990 ( 0.990)	Data  0.452 ( 0.452)	Loss 6.3718e+00 (6.3718e+00)	Acc@1   0.00 (  0.00)	Acc@5   0.00 (  0.00)
Epoch: [0][   63/20019]	Time  0.027 ( 0.058)	Data  0.000 ( 0.026)	Loss 1.9547e+01 (5.8396e+00)	Acc@1   0.00 ( 58.91)	Acc@5   0.00 ( 83.72)
Epoch: [0][  126/20019]	Time  0.175 ( 0.057)	Data  0.158 ( 0.029)	Loss 7.4909e+00 (6.5230e+00)	Acc@1   0.00 ( 34.77)	Acc@5   0.00 ( 79.65)
Epoch: [0][  189/20019]	Time  0.027 ( 0.060)	Data  0.000 ( 0.033)	Loss 6.7297e+00 (6.3152e+00)	Acc@1   0.00 ( 23.32)	Acc@5   0.00 ( 69.38)
Epoch: [0][  252/20019]	Time  0.396 ( 0.063)	Data  0.378 ( 0.036)	Loss 6.7017e+00 (6.2273e+00)	Acc@1   0.00 ( 17.51)	Acc@5   0.00 ( 56.60)
Epoch: [0][  315/20019]	Time  0.027 ( 0.064)	Data  0.000 ( 0.038)	Loss 6.2310e+00 (6.2284e+00)	Acc@1   0.00 ( 14.02)	Acc@5   0.00 ( 45.39)
Epoch: [0][  378/20019]	Time  0.280 ( 0.065)	Data  0.262 ( 0.039)	Loss 5.6553e+00 (6.2116e+00)	Acc@1   0.00 ( 11.69)	Acc@5   0.00 ( 37.85)
Epoch: [0][  441/20019]	Tim

Epoch: [0][ 3717/20019]	Time  0.028 ( 0.070)	Data  0.000 ( 0.046)	Loss 7.0986e+00 (6.6181e+00)	Acc@1   0.00 (  1.19)	Acc@5   0.00 (  3.86)
Epoch: [0][ 3780/20019]	Time  0.027 ( 0.070)	Data  0.000 ( 0.046)	Loss 6.8767e+00 (6.6223e+00)	Acc@1   0.00 (  1.17)	Acc@5   0.00 (  3.79)
Epoch: [0][ 3843/20019]	Time  0.027 ( 0.070)	Data  0.000 ( 0.046)	Loss 6.6552e+00 (6.6261e+00)	Acc@1   0.00 (  1.15)	Acc@5   0.00 (  3.73)
Epoch: [0][ 3906/20019]	Time  0.156 ( 0.070)	Data  0.139 ( 0.046)	Loss 6.7413e+00 (6.6299e+00)	Acc@1   0.00 (  1.13)	Acc@5   0.00 (  3.67)
Epoch: [0][ 3969/20019]	Time  0.027 ( 0.070)	Data  0.000 ( 0.046)	Loss 7.1012e+00 (6.6333e+00)	Acc@1   0.00 (  1.12)	Acc@5   0.00 (  3.61)
Epoch: [0][ 4032/20019]	Time  0.187 ( 0.070)	Data  0.170 ( 0.046)	Loss 7.2119e+00 (6.6368e+00)	Acc@1   0.00 (  1.10)	Acc@5   0.00 (  3.56)
Epoch: [0][ 4095/20019]	Time  0.030 ( 0.070)	Data  0.000 ( 0.046)	Loss 7.0056e+00 (6.6400e+00)	Acc@1   0.00 (  1.08)	Acc@5   0.00 (  3.50)
Epoch: [0][ 4158/20019]	Tim

Epoch: [0][ 7434/20019]	Time  0.027 ( 0.069)	Data  0.000 ( 0.045)	Loss 6.6433e+00 (6.7219e+00)	Acc@1   0.00 (  0.60)	Acc@5   0.00 (  1.93)
Epoch: [0][ 7497/20019]	Time  0.027 ( 0.069)	Data  0.000 ( 0.045)	Loss 7.2017e+00 (6.7234e+00)	Acc@1   0.00 (  0.59)	Acc@5   0.00 (  1.91)
Epoch: [0][ 7560/20019]	Time  0.027 ( 0.069)	Data  0.000 ( 0.045)	Loss 7.1093e+00 (6.7250e+00)	Acc@1   0.00 (  0.59)	Acc@5   0.00 (  1.90)
Epoch: [0][ 7623/20019]	Time  0.029 ( 0.069)	Data  0.000 ( 0.045)	Loss 7.1318e+00 (6.7264e+00)	Acc@1   0.00 (  0.58)	Acc@5   0.00 (  1.88)
Epoch: [0][ 7686/20019]	Time  0.026 ( 0.069)	Data  0.000 ( 0.045)	Loss 6.9806e+00 (6.7271e+00)	Acc@1   0.00 (  0.58)	Acc@5   0.00 (  1.87)
Epoch: [0][ 7749/20019]	Time  0.027 ( 0.069)	Data  0.000 ( 0.045)	Loss 6.9562e+00 (6.7281e+00)	Acc@1   0.00 (  0.57)	Acc@5   0.00 (  1.85)
Epoch: [0][ 7812/20019]	Time  0.027 ( 0.069)	Data  0.000 ( 0.045)	Loss 6.8576e+00 (6.7291e+00)	Acc@1   0.00 (  0.57)	Acc@5   0.00 (  1.84)
Epoch: [0][ 7875/20019]	Tim

Epoch: [0][11151/20019]	Time  0.027 ( 0.069)	Data  0.000 ( 0.045)	Loss 6.9653e+00 (6.7887e+00)	Acc@1   0.00 (  0.40)	Acc@5   0.00 (  1.29)
Epoch: [0][11214/20019]	Time  0.188 ( 0.069)	Data  0.170 ( 0.045)	Loss 7.0055e+00 (6.7895e+00)	Acc@1   0.00 (  0.40)	Acc@5   0.00 (  1.28)
Epoch: [0][11277/20019]	Time  0.028 ( 0.069)	Data  0.000 ( 0.045)	Loss 6.9552e+00 (6.7905e+00)	Acc@1   0.00 (  0.39)	Acc@5   0.00 (  1.27)
Epoch: [0][11340/20019]	Time  0.067 ( 0.069)	Data  0.050 ( 0.045)	Loss 7.0017e+00 (6.7914e+00)	Acc@1   0.00 (  0.39)	Acc@5   0.00 (  1.26)
Epoch: [0][11403/20019]	Time  0.026 ( 0.069)	Data  0.000 ( 0.045)	Loss 6.9651e+00 (6.7924e+00)	Acc@1   0.00 (  0.39)	Acc@5   0.00 (  1.26)
Epoch: [0][11466/20019]	Time  0.027 ( 0.069)	Data  0.000 ( 0.045)	Loss 6.9121e+00 (6.7931e+00)	Acc@1   0.00 (  0.39)	Acc@5   0.00 (  1.25)
Epoch: [0][11529/20019]	Time  0.026 ( 0.069)	Data  0.000 ( 0.045)	Loss 6.9263e+00 (6.7940e+00)	Acc@1   0.00 (  0.38)	Acc@5   0.00 (  1.24)
Epoch: [0][11592/20019]	Tim

Epoch: [0][14868/20019]	Time  0.026 ( 0.068)	Data  0.001 ( 0.044)	Loss 7.0588e+00 (6.8340e+00)	Acc@1   0.00 (  0.30)	Acc@5   0.00 (  0.96)
Epoch: [0][14931/20019]	Time  0.027 ( 0.068)	Data  0.000 ( 0.044)	Loss 7.0407e+00 (6.8346e+00)	Acc@1   0.00 (  0.30)	Acc@5   0.00 (  0.96)
Epoch: [0][14994/20019]	Time  0.257 ( 0.068)	Data  0.241 ( 0.044)	Loss 7.0337e+00 (6.8353e+00)	Acc@1   0.00 (  0.30)	Acc@5   0.00 (  0.96)
Epoch: [0][15057/20019]	Time  0.027 ( 0.068)	Data  0.000 ( 0.044)	Loss 7.0017e+00 (6.8359e+00)	Acc@1   0.00 (  0.29)	Acc@5   0.00 (  0.95)
Epoch: [0][15120/20019]	Time  0.127 ( 0.068)	Data  0.110 ( 0.044)	Loss 6.9613e+00 (6.8365e+00)	Acc@1   0.00 (  0.29)	Acc@5   0.00 (  0.95)
Epoch: [0][15183/20019]	Time  0.079 ( 0.068)	Data  0.062 ( 0.044)	Loss 6.9773e+00 (6.8372e+00)	Acc@1   0.00 (  0.29)	Acc@5   0.00 (  0.94)
Epoch: [0][15246/20019]	Time  0.026 ( 0.068)	Data  0.000 ( 0.044)	Loss 6.9147e+00 (6.8378e+00)	Acc@1   0.00 (  0.29)	Acc@5   0.00 (  0.94)
Epoch: [0][15309/20019]	Tim

Epoch: [0][18585/20019]	Time  0.261 ( 0.068)	Data  0.245 ( 0.044)	Loss 7.0760e+00 (6.8670e+00)	Acc@1   0.00 (  0.24)	Acc@5   0.00 (  0.77)
Epoch: [0][18648/20019]	Time  0.030 ( 0.068)	Data  0.000 ( 0.044)	Loss 7.0094e+00 (6.8675e+00)	Acc@1   0.00 (  0.24)	Acc@5   0.00 (  0.77)
Epoch: [0][18711/20019]	Time  0.259 ( 0.068)	Data  0.242 ( 0.044)	Loss 7.0634e+00 (6.8679e+00)	Acc@1   0.00 (  0.24)	Acc@5   0.00 (  0.77)
Epoch: [0][18774/20019]	Time  0.026 ( 0.068)	Data  0.000 ( 0.044)	Loss 7.0213e+00 (6.8684e+00)	Acc@1   0.00 (  0.24)	Acc@5   0.00 (  0.76)
Epoch: [0][18837/20019]	Time  0.273 ( 0.069)	Data  0.255 ( 0.044)	Loss 6.9624e+00 (6.8689e+00)	Acc@1   0.00 (  0.24)	Acc@5   0.00 (  0.76)
Epoch: [0][18900/20019]	Time  0.026 ( 0.069)	Data  0.000 ( 0.044)	Loss 6.9828e+00 (6.8693e+00)	Acc@1   0.00 (  0.23)	Acc@5   0.00 (  0.76)
Epoch: [0][18963/20019]	Time  0.188 ( 0.069)	Data  0.172 ( 0.044)	Loss 7.0095e+00 (6.8699e+00)	Acc@1   0.00 (  0.23)	Acc@5   0.00 (  0.76)
Epoch: [0][19026/20019]	Tim